# The First Part: Built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name

In [1]:
# import all the possibilly needed libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!pip install geopy
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium
import folium

print('Libraries imported.')

     |████████████████████████████████| 92kB 7.3MB/s eta 0:00:011
Libraries imported.


### Read the table to a dataframe

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Change the columns' name to meet the requirement

In [6]:
df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)
print(df.shape)
df.head()

(210, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### Only process the cells that have an assigned borough

In [3]:
df[df['Borough'] == 'Not assigned'].count() # checking how many borough with a "Not assigned" value

Postcode         77
Borough          77
Neighbourhood    77
dtype: int64

In [4]:
df = df[df.Borough != 'Not assigned']
print(df.shape) # Check if all the "Not assigned" values are deleted correctly
df.head(10)

(210, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


### Change the Neighborhood with a "Not assigned" value to the same value as the Borough.

In [7]:
# first checking how many Neighborhood with a "Not assigned" value
df[df['Neighborhood'] == 'Not assigned'].count()

PostalCode      1
Borough         1
Neighborhood    1
dtype: int64

In [8]:
df['Neighborhood'] = df['Borough'].where(df['Neighborhood'] == 'Not assigned', other = df['Neighborhood'])
df[df['Neighborhood'] == 'Not assigned'].count() # Check if the value has been changed

PostalCode      0
Borough         0
Neighborhood    0
dtype: int64

### Group postalcode and combine the relevent neighborhoods value separated with a comma

PS: The M5A example in the assighment is wrong. M5A is only listed once in the table on the Wikipedia page!

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [10]:
# df.groupby('PostalCode')['Neighborhood'].agg(','.join).reset_index()
df_grouped = df.groupby('PostalCode').agg({'Borough': lambda x: x.iloc[0], 'Neighborhood': ', '.join}).reset_index()
df_grouped.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [11]:
df_grouped.shape

(103, 3)

### Print the number of rows of my dataframe

In [12]:
print('The number of rows of my dataframe: ', df_grouped.shape[0])

The number of rows of my dataframe:  103


# The Second Part: Add the latitude and the longitude coordinates of each neighborhood

### Get the geographical coordinates of the neighborhoods using the csv file 

In [13]:
df_Geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df_Geo.shape

(103, 3)

In [15]:
df_Geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_Geo.head(20)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merge the two dataframe based on the PostalCode

In [16]:
df_nh = pd.merge(df_grouped, df_Geo, on = 'PostalCode')
print(df_nh.shape)
df_nh.head(15)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# The Third/Last Part: Explore and cluster the neighborhoods in Toronto

### Check the unique values in Borough

In [21]:
df_nh['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           11
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

### Use geopy library to get the latitude and longitude values of Toronto.

In [41]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer", timeout=3)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with neighborhoods superimposed on top

In [35]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_nh['Latitude'], df_nh['Longitude'], df_nh['Borough'], df_nh['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Work with only boroughs that contain the word Toronto

In [33]:
df_conToronto = df_nh[df_nh['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(df_conToronto.shape) # should be 39 rows based on the Borough's unique value check
df_conToronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [37]:
map_conToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_conToronto['Latitude'], df_conToronto['Longitude'], df_conToronto['Borough'], df_conToronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_conToronto)  
    
map_conToronto

### Work with North York

In [40]:
df_ny = df_nh[df_nh['Borough'] == 'North York'].reset_index(drop=True)
print(df_ny.shape)
df_ny.head()

(24, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [43]:
address = 'North York'

geolocator = Nominatim(user_agent="ny_explorer", timeout=3)
location = geolocator.geocode(address)
latitude_ny = location.latitude
longitude_ny = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude_ny, longitude_ny))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [44]:
map_ny = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ny['Latitude'], df_ny['Longitude'], df_ny['Borough'], df_ny['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

### Utilizing the Foursquare API to explore the neighborhoods and segment them. First define Foursquare Credentials and Version

In [46]:
CLIENT_ID = 'QXTB3NFSILI2SK3VTC5ECYRV0LIEWY5L42ZJKQBGBCXVHVST'
CLIENT_SECRET = '3XMITHOMQZ5GF3LLFLAFS22KGU5V2YDVAXZLMEYGMGGW2KYB'
VERSION = '20200123' 

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

CLIENT_ID: QXTB3NFSILI2SK3VTC5ECYRV0LIEWY5L42ZJKQBGBCXVHVST
CLIENT_SECRET: 3XMITHOMQZ5GF3LLFLAFS22KGU5V2YDVAXZLMEYGMGGW2KYB


## 1. Explore the first neighborhood in Toronto.

In [47]:
df_nh.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [48]:
neighborhood_latitude = df_nh.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_nh.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_nh.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


### get the top 100 venues that are in the first neighborhood in Toronto within a radius of 1000 meters.

In [57]:
radius=1000
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

In [58]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [59]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy's,Fast Food Restaurant,43.802008,-79.198080
2,Wendy's,Fast Food Restaurant,43.807448,-79.199056
3,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
4,Tim Hortons,Coffee Shop,43.802000,-79.198169


In [60]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


## 2. Explore Neighborhoods in North York

In [61]:
# create a function to repeat the same process to all the neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
ny_venues = getNearbyVenues(names=df_ny['Neighborhood'],
                                   latitudes=df_ny['Latitude'],
                                   longitudes=df_ny['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [63]:
print(ny_venues.shape)
ny_venues.head()

(238, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store


In [64]:
ny_venues.groupby('Neighborhood').count() # check how many venues were returned for each neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


## 3. Analyze Each Neighborhood in North York

In [65]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [66]:
ny_onehot.shape

(238, 104)

In [67]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
print(ny_grouped.shape)
ny_grouped.head()

(23, 104)


,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Business Service,Butcher,Cafeteria,Café,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shopping Mall,Smoke Shop,Snack Place,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.052632,0.0,0.000000,0.000000,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.0,0.0,0.105263,0.00,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.052632,0.052632,0.0,0.052632,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.052632,0.00,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.000000,0.0,0.052632,0.052632,0.0,0.0,0.0,0.00,0.0,0.052632,0.0,0.052632,0.052632,0.0,0.000000,0.0,0.0,0.052632,0.0,0.052632,0.0,0.0,0.00,0.0,0.00,0.0,0.052632,0.0,0.0,0.00
1,Bayview Village,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.250000,0.000000,0.25,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.250000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.04,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.04,0.0,0.040000,0.000000,0.00,0.0,0.0,0.080000,0.04,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.04,0.04,0.0,0.000000,0.0,0.0,0.000000,0.04,0.0,0.08,0.000000,0.04,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.040000,0.040000,0.0,0.0,0.0,0.04,0.0,0.040000,0.0,0.080000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.040000,0.0,0.0,0.04,0.0,0.04,0.0,0.000000,0.0,0.0,0.04
3,"CFB Toronto, Downsview East",0.0,0.333333,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.0,0.00,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.00
4,Don Mills North,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0

### write a function to sort the venues in descending order

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 10 venues for each neighborhood.

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Women's Store,Pub,Pizza Place,Pharmacy,Liquor Store,Juice Bar
3,"CFB Toronto, Downsview East",Airport,Park,Snack Place,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills North,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store
5,Downsview Central,Korean Restaurant,Food Truck,Business Service,Baseball Field,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
6,Downsview Northwest,Grocery Store,Liquor Store,Gym / Fitness Center,Athletics & Sports,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
7,Downsview West,Grocery Store,Park,Hotel,Bank,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
8,"Downsview, North Park, Upwood Park",Park,Construction & Landscaping,Bakery,Basketball Court,Women's Store,Dog Run,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega
9,"Emery, Humberlea",Baseball Field,Women's Store,Dog Run,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## 4. Cluster Neighborhoods

In [94]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 1, 3, 1, 4], dtype=int32)

In [95]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = df_ny

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1.0,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1.0,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Electronics Store,Japanese Restaurant,Juice Bar,Women's Store,Toy / Game Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,1.0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2.0,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M2N,North York,Willowdale South,43.770120,-79.408493,1.0,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Coffee Shop,Restaurant,Café,Pizza Place,Indonesian Restaurant,Ice Cream Shop,Bubble Tea Shop
6,M2P,North York,York Mills West,43.752758,-79.400049,3.0,Park,Moving Target,Convenience Store,Bank,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
7,M2R,North York,Willowdale West,43.782736,-79.442259,1.0,Grocery Store,Butcher,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Athletics & Sports,Dog Run,Concert Hall,Construction & Landscaping
8,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Pool,Park,Food & Drink Shop,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
9,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store


In [96]:
ny_merged = ny_merged[np.isfinite(ny_merged['Cluster Labels'])].reset_index()
ny_merged['Cluster Labels']= ny_merged['Cluster Labels'].astype(int)
ny_merged

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
1,1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Electronics Store,Japanese Restaurant,Juice Bar,Women's Store,Toy / Game Store
2,2,M2K,North York,Bayview Village,43.786947,-79.385975,1,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
3,3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,2,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
4,5,M2N,North York,Willowdale South,43.770120,-79.408493,1,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Coffee Shop,Restaurant,Café,Pizza Place,Indonesian Restaurant,Ice Cream Shop,Bubble Tea Shop
5,6,M2P,North York,York Mills West,43.752758,-79.400049,3,Park,Moving Target,Convenience Store,Bank,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
6,7,M2R,North York,Willowdale West,43.782736,-79.442259,1,Grocery Store,Butcher,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Athletics & Sports,Dog Run,Concert Hall,Construction & Landscaping
7,8,M3A,North York,Parkwoods,43.753259,-79.329656,0,Pool,Park,Food & Drink Shop,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
8,9,M3B,North York,Don Mills North,43.745906,-79.352188,1,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store
9,10,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,1,Coffee Shop,Asian Restaurant,Gym,Beer Store,Sporting Goods Shop,Bike Shop,Clothing Store,Concert Hall,Chinese Restaurant,Dim Sum Restaurant


### visualize the resulting clusters

In [97]:
# create map
map_clusters = folium.Map(location=[latitude_ny, longitude_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Check each Clusters

In [99]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M3A,-79.329656,0,Pool,Park,Food & Drink Shop,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


In [100]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,-79.363452,1,Golf Course,Pool,Mediterranean Restaurant,Dog Run,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
1,M2J,-79.346556,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Bakery,Food Court,Electronics Store,Japanese Restaurant,Juice Bar,Women's Store,Toy / Game Store
2,M2K,-79.385975,1,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,M2N,-79.408493,1,Ramen Restaurant,Sushi Restaurant,Sandwich Place,Coffee Shop,Restaurant,Café,Pizza Place,Indonesian Restaurant,Ice Cream Shop,Bubble Tea Shop
6,M2R,-79.442259,1,Grocery Store,Butcher,Discount Store,Coffee Shop,Pharmacy,Pizza Place,Athletics & Sports,Dog Run,Concert Hall,Construction & Landscaping
8,M3B,-79.352188,1,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Basketball Court,Japanese Restaurant,Women's Store,Dog Run,Construction & Landscaping,Convenience Store
9,M3C,-79.340923,1,Coffee Shop,Asian Restaurant,Gym,Beer Store,Sporting Goods Shop,Bike Shop,Clothing Store,Concert Hall,Chinese Restaurant,Dim Sum Restaurant
10,M3H,-79.442259,1,Coffee Shop,Fried Chicken Joint,Supermarket,Pharmacy,Pizza Place,Deli / Bodega,Bridal Shop,Restaurant,Diner,Sandwich Place
11,M3J,-79.487262,1,Caribbean Restaurant,Miscellaneous Shop,Bar,Massage Studio,Coffee Shop,Women's Store,Diner,Concert Hall,Construction & Landscaping,Convenience Store
14,M3M,-79.495697,1,Korean Restaurant,Food Truck,Business Service,Baseball Field,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [101]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M2L,-79.374714,2,Cafeteria,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


In [102]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,M2P,-79.400049,3,Park,Moving Target,Convenience Store,Bank,Women's Store,Discount Store,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
12,M3K,-79.464763,3,Airport,Park,Snack Place,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
13,M3L,-79.506944,3,Grocery Store,Park,Hotel,Bank,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [103]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,M9M,-79.532242,4,Baseball Field,Women's Store,Dog Run,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
